In [1]:
import taichi as ti

[Taichi] version 1.0.0, llvm 10.0.0, commit 6a15da85, linux, python 3.9.7
[I 04/13/22 15:01:40.068 41565] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout


In [2]:
ti.init(arch=ti.gpu)

[Taichi] Starting on arch=opengl
[W 04/13/22 15:01:41.192 41565] [cuda_driver.cpp:CUDADriver@38] CUDA driver not found.


In [3]:
vec3f = ti.types.vector(3, ti.f32)
mat2f = ti.types.matrix(2,2, ti.f32)
ray = ti.types.struct(ro=vec3f, rd=vec3f, l=ti.f32)

@ti.kernel
def foo():
    a = vec3f(0.0) # origin
    print(a)
    d = vec3f(0.0, 1.0, 0.0) # direction
    print(d)
    B = mat2f([[1.5, 1.4], [1.3,1.2]])
    print("B = ",B)
    r = ray(ro=a, rd=d, l=1)
    print("r.ro =", r.ro)
    print("r.rd =", r.rd)
foo()

[E 04/12/22 12:55:22.672 60424] [opengl_device.cpp:GLPipeline@257] [glsl] error while compiling shader:
0:1(10): error: GLSL 4.30 is not supported. Supported versions are: 1.10, 1.20, 1.30, 1.00 ES, and 3.00 ES
0:0(0): error: Compute shaders require GLSL 4.30 or GLSL ES 3.10



***********************************
* Taichi Compiler Stack Traceback *
***********************************
/home/zichen/Programs/anaconda3/lib/python3.9/site-packages/taichi/_lib/core/taichi_core.so: taichi::Logger::error(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, bool)
/home/zichen/Programs/anaconda3/lib/python3.9/site-packages/taichi/_lib/core/taichi_core.so(+0xa8fae4) [0x7f5ff4aedae4]
/home/zichen/Programs/anaconda3/lib/python3.9/site-packages/taichi/_lib/core/taichi_core.so(+0xa92243) [0x7f5ff4af0243]
/home/zichen/Programs/anaconda3/lib/python3.9/site-packages/taichi/_lib/core/taichi_core.so(+0xa8b931) [0x7f5ff4ae9931]
/home/zichen/Programs/anaconda3/lib/python3.

RuntimeError: [opengl_device.cpp:GLPipeline@257] [glsl] error while compiling shader:
0:1(10): error: GLSL 4.30 is not supported. Supported versions are: 1.10, 1.20, 1.30, 1.00 ES, and 3.00 ES
0:0(0): error: Compute shaders require GLSL 4.30 or GLSL ES 3.10


In [3]:
# global control
paused = ti.field(ti.i32, ())

# gravitational constant 6.67408e-11, using 1 for simplicity
G = 1
PI = 3.141592653

# number of planets
N = 1000

# unit mass
m=5

# galaxy size
galaxy_size=0.4

# planet radius (for rendering)
planet_radius = 2

# init vel
init_vel = 120

# time-step size
h = 1e-5

# substepping
substepping = 10

# pos, vel and force of the planets
# Nx2 vectors
pos = ti.Vector.field(2, ti.f32, N)
vel = ti.Vector.field(2, ti.f32, N)
force = ti.Vector.field(2, ti.f32, N)

@ti.kernel
def initialize():
    center = ti.Vector([0.5, 0.5])
    for i in range(N):
        theta = ti.random()*2*PI
        r = (ti.sqrt(ti.random())*0.7 + 0.3) * galaxy_size
        offset = r* ti.Vector([ti.cos(theta), ti.sin(theta)])
        pos[i] = center + offset
        vel[i] = [-offset.y, offset.x]
        vel[i] *= init_vel

@ti.kernel
def compute_force():
    # clear force
    for i in range(N):
        force[i] = ti.Vector([0.0, 0.0])
    
    # compute gravitational force
    for i in range(N):
        p = pos[i]
        for j in range(i):
            diff = p-pos[j]
            r = diff.norm(1e-5)
            
            # gravitational force -(GMm/r^2)*(diff/r) for i
            f = -G*m*m*(1.0/r)**3 * diff
            
            # assign to each particle
            force[i] +=f
            force[j] += -f
            
@ti.kernel
def update():
    dt = h/substepping
    for i in range(N):
        # sympletic euler
        vel[i] += dt*force[i]/m
        pos[i] += dt*vel[i]

gui = ti.GUI('N-body problem', (512,512))
# gui = ti.GUI(show_gui=False)

initialize()
while gui.running:
    for i in range(substepping):
        compute_force()
        update()
        
    gui.clear(0x112F41)
    gui.circles(pos.to_numpy(), color=0xffffff, radius=planet_radius)
    gui.show()

[E 04/13/22 15:02:43.922 41565] [opengl_device.cpp:GLPipeline@257] [glsl] error while compiling shader:
0:1(10): error: GLSL 4.30 is not supported. Supported versions are: 1.10, 1.20, 1.30, 1.00 ES, and 3.00 ES
0:0(0): error: Compute shaders require GLSL 4.30 or GLSL ES 3.10



***********************************
* Taichi Compiler Stack Traceback *
***********************************
/home/zichen/Programs/anaconda3/envs/nerf-pytorch/lib/python3.9/site-packages/taichi/_lib/core/taichi_core.so: taichi::Logger::error(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, bool)
/home/zichen/Programs/anaconda3/envs/nerf-pytorch/lib/python3.9/site-packages/taichi/_lib/core/taichi_core.so(+0xa9da24) [0x7fe685847a24]
/home/zichen/Programs/anaconda3/envs/nerf-pytorch/lib/python3.9/site-packages/taichi/_lib/core/taichi_core.so(+0xaa0183) [0x7fe68584a183]
/home/zichen/Programs/anaconda3/envs/nerf-pytorch/lib/python3.9/site-packages/taichi/_lib/core/taichi_core.so(

RuntimeError: [opengl_device.cpp:GLPipeline@257] [glsl] error while compiling shader:
0:1(10): error: GLSL 4.30 is not supported. Supported versions are: 1.10, 1.20, 1.30, 1.00 ES, and 3.00 ES
0:0(0): error: Compute shaders require GLSL 4.30 or GLSL ES 3.10
